In [19]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Dropout
from keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
from keras.callbacks import EarlyStopping, ModelCheckpoint
# from keras.wrappers.scikit_learn import KerasClassifier
from math import floor
from sklearn.metrics import make_scorer, accuracy_score
from bayes_opt import BayesianOptimization, UtilityFunction
from sklearn.model_selection import StratifiedKFold
from keras.layers import LeakyReLU
LeakyReLU = LeakyReLU(alpha=0.1)
import warnings
warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", None)
from scikeras.wrappers import KerasClassifier

In [29]:
score_acc = make_scorer(accuracy_score)

In [34]:
score_acc

make_scorer(accuracy_score, response_method='predict')

In [13]:
trainSet = pd.read_csv(r'C:\Users\jv028u\OneDrive - Linde Group\Desktop\stats and ML\Projects\train.csv')

train = trainSet.drop(columns=['Name', 'Ticket', 'Cabin'])
train = train.dropna(axis=0)
train = pd.get_dummies(train)



In [ ]:
X = train.drop(columns=['PassengerId', 'Survived'])
y = train['Survived']

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=111, stratify=y
)

In [31]:
def nn_cl_bo(neurons, activation, optimizer, learning_rate, batch_size, epochs):
    optimizerL = ['SGD', 'Adam', 'RMSprop', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Ftrl']
    optimizerD = {
        'Adam': Adam(learning_rate=learning_rate), 
        'SGD': SGD(learning_rate=learning_rate),
        'RMSprop': RMSprop(learning_rate=learning_rate), 
        'Adadelta': Adadelta(learning_rate=learning_rate),
        'Adagrad': Adagrad(learning_rate=learning_rate), 
        'Adamax': Adamax(learning_rate=learning_rate),
        'Nadam': Nadam(learning_rate=learning_rate), 
        'Ftrl': Ftrl(learning_rate=learning_rate)
    }
    activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu',
                   'elu', 'exponential', LeakyReLU, 'relu']
    
    neurons = round(neurons)
    activation = activationL[round(activation)]
    batch_size = round(batch_size)
    epochs = round(epochs)

    def nn_cl_fun():
        opt = Adam(learning_rate=learning_rate)
        nn = Sequential()
        nn.add(Dense(neurons, input_dim=10, activation=activation))
        nn.add(Dense(neurons, activation=activation))
        nn.add(Dense(1, activation='sigmoid'))
        nn.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
        return nn

    es = EarlyStopping(monitor='accuracy', mode='max', verbose=0, patience=20)
    nn = KerasClassifier(build_fn=nn_cl_fun, epochs=epochs, batch_size=batch_size, verbose=0)
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)
    score = cross_val_score(nn, X_train, y_train, scoring='accuracy', cv=kfold, fit_params={'callbacks': [es]}).mean()
    return score

In [32]:

params_nn ={
    'neurons': (10, 100),
    'activation':(0, 9),
    'optimizer':(0,7),
    'learning_rate':(0.01, 1),
    'batch_size':(200, 1000),
    'epochs':(20, 100)
}
# Run Bayesian Optimization
nn_bo = BayesianOptimization(nn_cl_bo, params_nn, random_state=111)
nn_bo.maximize(init_points=25, n_iter=4)

|   iter    |  target   | activa... | batch_... |  epochs   | learni... |  neurons  | optimizer |
-------------------------------------------------------------------------------------------------
| 1         | 0.5144    | 5.51      | 335.3     | 54.88     | 0.7716    | 36.58     | 1.044     |
| 2         | 0.5431    | 0.2023    | 536.2     | 39.09     | 0.3443    | 99.16     | 1.664     |
| 3         | 0.6079    | 0.7307    | 735.7     | 69.7      | 0.2815    | 51.96     | 0.8286    |
| 4         | 0.5431    | 0.6656    | 920.6     | 83.52     | 0.8422    | 83.37     | 6.937     |
| 5         | 0.7639    | 5.195     | 851.0     | 53.71     | 0.03717   | 50.87     | 0.7373    |
| 6         | 0.5719    | 7.355     | 758.2     | 65.22     | 0.2815    | 99.86     | 0.9663    |
| 7         | 0.5719    | 5.539     | 588.0     | 52.4      | 0.7306    | 39.05     | 2.804     |
| 8         | 0.6115    | 2.871     | 957.8     | 93.5      | 0.8157    | 13.07     | 6.604     |
| 9         | 0.5719

In [39]:
params_nn_ = nn_bo.max['params']
activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu',
               'elu', 'exponential', LeakyReLU,'relu']
best_params['activation'] = activationL[round(best_params['activation'])]

optimizerL = ['SGD', 'Adam', 'RMSprop', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Ftrl']
optimizers = {
    'Adam': Adam(learning_rate=best_params['learning_rate']), 
    'SGD': SGD(learning_rate=best_params['learning_rate']),
    'RMSprop': RMSprop(learning_rate=best_params['learning_rate']), 
    'Adadelta': Adadelta(learning_rate=best_params['learning_rate']),
    'Adagrad': Adagrad(learning_rate=best_params['learning_rate']), 
    'Adamax': Adamax(learning_rate=best_params['learning_rate']),
    'Nadam': Nadam(learning_rate=best_params['learning_rate']), 
    'Ftrl': Ftrl(learning_rate=best_params['learning_rate'])
}
best_optimizer = optimizers[optimizerL[round(best_params['optimizer'])]]

# Build the model with the best hyperparameters
def build_optimized_nn():
    nn = Sequential()
    nn.add(Dense(units=round(best_params['neurons']), input_dim=X_train.shape[1], activation=best_params['activation']))
    nn.add(Dense(units=round(best_params['neurons']), activation=best_params['activation']))
    nn.add(Dense(1, activation='sigmoid'))  # Output layer for binary classification
    nn.compile(loss='binary_crossentropy', optimizer=best_optimizer, metrics=['accuracy'])
    return nn

# Early stopping callback
es = EarlyStopping(monitor='val_accuracy', mode='max', verbose=1, patience=10)

# Train the model with optimized parameters
model = KerasClassifier(build_fn=build_optimized_nn, 
                        epochs=round(best_params['epochs']), 
                        batch_size=round(best_params['batch_size']), 
                        verbose=1)

# Train the model
model.fit(X_train, y_train, validation_data=(X_val, y_val), callbacks=[es])

# Predict on the validation set
y_pred = model.predict(X_val)
y_pred = (y_pred > 0.5).astype(int)  # Convert probabilities to binary predictions

# Evaluate accuracy on the validation set
accuracy = accuracy_score(y_val, y_pred)
print(f"Validation Accuracy: {accuracy * 100:.2f}%")

{'activation': 'selu',
 'batch_size': 373.87633695429145,
 'epochs': 77.53809882694163,
 'learning_rate': 0.04056128946327648,
 'neurons': 47.68371572646683,
 'optimizer': 1.968575834097605}

In [37]:
best_params = nn_bo.max['params']
activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu', 'elu', 'exponential', LeakyReLU, 'relu']
best_params['activation'] = activationL[round(best_params['activation'])]


optimizerL = ['SGD', 'Adam', 'RMSprop', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Ftrl']
optimizers = {
    'Adam': Adam(learning_rate=best_params['learning_rate']), 
    'SGD': SGD(learning_rate=best_params['learning_rate']),
    'RMSprop': RMSprop(learning_rate=best_params['learning_rate']), 
    'Adadelta': Adadelta(learning_rate=best_params['learning_rate']),
    'Adagrad': Adagrad(learning_rate=best_params['learning_rate']), 
    'Adamax': Adamax(learning_rate=best_params['learning_rate']),
    'Nadam': Nadam(learning_rate=best_params['learning_rate']), 
    'Ftrl': Ftrl(learning_rate=best_params['learning_rate'])
}


best_optimizer = optimizers[optimizerL[round(best_params['optimizer'])]]

# Build the model with optimized hyperparameters
def build_optimized_nn():
    nn = Sequential()
    nn.add(Dense(units=round(best_params['neurons']), input_dim=X_train.shape[1], activation=best_params['activation']))
    nn.add(Dense(units=round(best_params['neurons']), activation=best_params['activation']))
    nn.add(Dense(1, activation='sigmoid'))  # Output layer for binary classification
    nn.compile(loss='binary_crossentropy', optimizer=best_optimizer, metrics=['accuracy'])
    return nn

# Define the early stopping callback
es = EarlyStopping(monitor='val_accuracy', mode='max', verbose=1, patience=10)

# Initialize KerasClassifier with the optimized parameters
model = KerasClassifier(build_fn=build_optimized_nn, 
                        epochs=round(best_params['epochs']), 
                        batch_size=round(best_params['batch_size']), 
                        verbose=1)

# Train the model
model.fit(X_train, y_train, validation_data=(X_val, y_val), callbacks=[es])

# Predict on the validation set
y_pred = model.predict(X_val)
y_pred = (y_pred > 0.5).astype(int)  # Convert probabilities to binary predictions

# Evaluate accuracy on the validation set
accuracy = accuracy_score(y_val, y_pred)
print(f"Validation Accuracy: {accuracy * 100:.2f}%")

Epoch 1/78
207/207 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.5152 - loss: 4.9662 - val_accuracy: 0.6033 - val_loss: 0.9178
Epoch 2/78
207/207 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.5187 - loss: 1.0720 - val_accuracy: 0.5719 - val_loss: 0.9530
Epoch 3/78
207/207 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.5801 - loss: 0.9513 - val_accuracy: 0.7304 - val_loss: 0.6878
Epoch 4/78
207/207 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6259 - loss: 0.8378 - val_accuracy: 0.7240 - val_loss: 0.6105
Epoch 5/78
207/207 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6366 - loss: 0.8719 - val_accuracy: 0.7230 - val_loss: 0.7088
Epoch 6/78
207/207 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6669 - loss: 0.7949 - val_accuracy: 0.7443 - val_loss: 0.5881
Epoch 7/78
207/207 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6671 - loss: 0.7959 - val_accuracy: 0.7593 - val_loss: 0.5340
Epoch 8/78
207/207 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6687 - loss: 0.7724 - val_accuracy: 0.